# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
import numpy as np
import sqlite3
import nltk
import string
import re 
    
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

In [2]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response')
df = pd.read_sql('SELECT * FROM Disaster_Response', engine)

X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
   
    # Convert text to lowercase and remove punctuation
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    
    # tokenize words
    tokens = nltk.word_tokenize(text)
   
    # lemmatize and remove stop words
    stop_words = nltk.corpus.stopwords.words("english")
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return lemmatized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# import libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

In [6]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier(n_estimators=10)))
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=101)

In [8]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
from sklearn.metrics import classification_report,f1_score
from scipy.stats.mstats import gmean

Y_pred = pipeline.predict(X_test)

for i, col in enumerate(Y_test):
    print('---------------------------------')
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

---------------------------------
related
             precision    recall  f1-score   support

          0       0.33      0.14      0.20      1862
          1       0.77      0.91      0.83      5946
          2       0.67      0.04      0.07        55

avg / total       0.66      0.72      0.68      7863

---------------------------------
request
             precision    recall  f1-score   support

          0       0.84      0.97      0.90      6546
          1       0.35      0.09      0.14      1317

avg / total       0.76      0.82      0.77      7863

---------------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7827
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7863

---------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.59      0.80      0.68      4599
          1       0.45   

In [10]:
def multi_f1score(y_true, y_pred): 
    scores = []
    for i in range(Y.shape[1]): 
        score = f1_score(y_true.iloc[:,i], y_pred[:,i],average='weighted')
        scores.append(score)
    scores = np.asarray(scores)
    score = gmean(scores)
    return score

multi_f1score(Y_test,Y_pred)

0.89126634132868487

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'clf__estimator__n_estimators':[50,100],
    'clf__estimator__min_samples_split':[2,4],
    'clf__estimator__criterion': ['entropy', 'gini']
}


cv = GridSearchCV(pipeline,param_grid=parameters,verbose = 2, n_jobs = -1)
cv.fit(X_train, Y_train)


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 4.5min
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  4.9min remaining:    0.0s


[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 4.5min
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50, total= 4.5min
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 8.7min
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100, total= 8.8min
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV]  clf__estimator__criterion=entropy, clf__estimator

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 156.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__n_estimators': [50, 100], 'clf__estimator__min_samples_split': [2, 4], 'clf__estimator__criterion': ['entropy', 'gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [13]:
Y_pred = cv.best_estimator_.predict(X_test)

multi_f1score(Y_test,Y_pred)

0.89188127699967434

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# import libraries
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [15]:
Y_pred = pipeline.predict(X_test)

for i, col in enumerate(Y_test):
    print('---------------------------------')
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i]))

multi_f1score(Y_test,Y_pred)

---------------------------------
related
             precision    recall  f1-score   support

          0       0.33      0.01      0.02      1862
          1       0.76      0.99      0.86      5946
          2       0.07      0.02      0.03        55

avg / total       0.65      0.75      0.65      7863

---------------------------------
request
             precision    recall  f1-score   support

          0       0.84      0.97      0.90      6546
          1       0.38      0.10      0.16      1317

avg / total       0.77      0.82      0.78      7863

---------------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7827
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      7863

---------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.60      0.87      0.71      4599
          1       0.48   

0.89145059405045957

### 9. Export your model as a pickle file

In [28]:
import pickle

pickle.dump(pipeline, open('../models/classifier.pkl', "wb"))

FileNotFoundError: [Errno 2] No such file or directory: '../models/classifier.pkl'

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.